In [1]:
import phrase_entity_extraction as ph

In [2]:
st = "W239 N1700 BUSSE ROAD".lower()
max_address = ph.return_max_address(ph.seq, st)
if st.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print("OH OH\n")
encoding = ph.encode_from_word_list(ph.w(st))
print()
print("Max Address: ", max_address)
print()
print("Encoding: ", encoding)
print('\n\n{},"{}"'.format(st.upper(), encoding))

ALL GOOD!

Max Address:  W239 N1700 BUSSE ROAD

Encoding:  [['ADR_HEAD', 'ALNUM'], ['ADR_HEAD', 'ALNUM'], ['ALPHA'], ['WORDWAY']]


W239 N1700 BUSSE ROAD,"[['ADR_HEAD', 'ALNUM'], ['ADR_HEAD', 'ALNUM'], ['ALPHA'], ['WORDWAY']]"


In [3]:
arr_cands = ph.get_markers(ph.seq, st, ['ADDRESS', 'POBOX'])
arr_cands

[[0, 4, 4, ['ADDRESS'], 'w239 n1700 busse road'],
 [1, 3, 2, ['ADDRESS'], 'n1700 busse'],
 [1, 4, 3, ['ADDRESS'], 'n1700 busse road']]

In [4]:
import re
def w(str_sentence):
    return re.findall(r"[\w'/\-:#]+|[,!?&]", str_sentence)

w(st)

['w239', 'n1700', 'busse', 'road']